<a href="https://colab.research.google.com/github/mohammad-hosein/Semi-Blind-MLP/blob/main/Copy_of_Semi_blind_MLP_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math
import pandas as pd
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

# Generating datasets and preprocessing

In [2]:
'''Fading channel for specific number of antennas at transmitter and receiver side'''
j=complex(0,1)
def fading_channel (Nt,Nr):
  h_real=np.random.normal(0, 0.5, size=(Nr, Nt))
  h_img=np.random.normal(0, 0.5, size=(Nr, Nt))
  H=np.array(h_real + h_img*j, dtype=complex)
  return H

In [3]:
'''SSK Constellation set'''
Constellation_set= np.zeros(shape= (4,4))
Constellation_set[0]= np.array([1,0,0,0])
Constellation_set[1]= np.array([0,1,0,0])
Constellation_set[2]= np.array([0,0,1,0])
Constellation_set[3]= np.array([0,0,0,1])

In [4]:
H=fading_channel (4,2)

In [5]:
'''Complex noise'''
def noise (N0,Nr):
  n_real=np.random.normal(0, N0, size=Nr)
  n_img=np.random.normal(0, N0, size=Nr)
  noise=np.array(n_real + n_img*j, dtype=complex)
  return noise

In [6]:
'''Creating pilot signals'''
def training (Label):
  i= 0
  y= np.zeros(shape= (4,2),dtype = "complex_")
  for i in range(Label):
    y[i]=np.dot(H, Constellation_set[i], out=None)+noise (0.16,2)
    i=i+1
  y_real=y.real
  y_imag=y.imag
  y1= np.concatenate((y_real,y_imag), axis=1)
  y2= pd.DataFrame(y1, columns=['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag'])
  y2['Label']=np.arange(0,4)
  return y2

In [7]:
training (4)

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.316899,-0.293157,-0.253901,0.186975,0
1,0.694534,-0.759647,-0.092526,-0.219142,1
2,-0.652949,-1.110681,0.039898,-0.392005,2
3,-0.484973,0.246818,-0.633100,-0.046310,3


In [8]:
'''Creating test signals'''
def test_ (size):
  y3=pd.DataFrame()
  for i in range(size):
    y= np.zeros(shape=(1,2),dtype = "complex_")
    rand=random.randrange(0, 4, 1)
    y=np.dot(H, Constellation_set[rand], out=None)+noise (0.16,2)
    y_real=y.real
    y_imag=y.imag
    y1= np.concatenate((y_real,y_imag), axis=None)
    y2=pd.DataFrame(y1.reshape((1,4)),columns=['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag'])
    if rand == 0 :
      y2['Label']= 0
    elif rand == 1 :
      y2['Label']= 1
    elif rand == 2 :
      y2['Label']= 2
    elif rand == 3 :
      y2['Label']= 3
    y3=pd.concat([y3,y2])
  return y3

In [9]:
test=test_ (2000)

In [10]:
test

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,0.823210,-0.782546,0.088219,-0.084552,1
0,0.969688,-0.680086,-0.197416,-0.103045,1
0,-0.842799,-1.122123,-0.379911,-0.161689,2
0,-0.496119,-0.809327,-0.310262,-0.255878,2
0,1.118004,-0.775014,-0.491793,-0.398065,1
...,...,...,...,...,...
0,-1.047957,-0.906322,-0.257034,-0.022564,2
0,0.158185,-0.555106,0.025738,0.169153,0
0,-0.342720,-0.470870,-0.262448,-0.028559,0
0,-0.194493,-0.260949,-0.333626,0.443090,0


In [11]:
m1=test[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
n1=test['Label']
m1_NN=test[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
n1_NN=test['Label']
n1_NN=to_categorical(n1, num_classes=4)

# Generating virtual pilots (data augmentation)

In [12]:
'''Noise for data augmentation'''
def noise_real (N0,Nr):
  n_real=np.random.normal(0, N0, size=Nr)
  return n_real

In [13]:
'''Data augmentation'''
q1=training (4)
l=pd.DataFrame()
for i in range(500):
  qs=q1[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]+noise_real(0.05,4)
  qs['Label']=np.arange(0,4)
  l = l.append(qs)
  i=i+1

In [14]:
print(q1.describe())

print(l.describe())


       Ant1_real  Ant2_real  Ant1_imag  Ant2_imag     Label
count   4.000000   4.000000   4.000000   4.000000  4.000000
mean   -0.019645  -0.568444  -0.220126  -0.049446  1.500000
std     0.662169   0.614284   0.300321   0.185175  1.290994
min    -0.516564  -1.211678  -0.584627  -0.205654  0.000000
25%    -0.329643  -0.936857  -0.382389  -0.158153  0.750000
50%    -0.259394  -0.642075  -0.206586  -0.103274  1.500000
75%     0.050604  -0.273662  -0.044323   0.005434  2.250000
max     0.956773   0.222052   0.117295   0.214419  3.000000
         Ant1_real    Ant2_real    Ant1_imag    Ant2_imag        Label
count  2000.000000  2000.000000  2000.000000  2000.000000  2000.000000
mean     -0.016668    -0.569007    -0.220037    -0.046059     1.500000
std       0.575701     0.534327     0.264637     0.167920     1.118314
min      -0.642539    -1.344844    -0.720377    -0.354724     0.000000
25%      -0.388254    -1.004036    -0.440725    -0.171828     0.750000
50%      -0.257153    -0.643267   

In [15]:
'''Data augmentation
q2=training (4)
i=0
for i in range(10):
  qs=q2[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.05,8)
  qs['Label']=np.arange(0,4)
  l = l.append(qs)
  i=i+1'''

"Data augmentation\nq2=training (4)\ni=0\nfor i in range(10):\n  qs=q2[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.05,8)\n  qs['Label']=np.arange(0,4)\n  l = l.append(qs)\n  i=i+1"

In [16]:
'''Data augmentation
q3=training (4)
i=0
for i in range(10):
  qs=q3[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.05,8)
  qs['Label']=np.arange(0,4)
  l = l.append(qs)
  i=i+1'''

"Data augmentation\nq3=training (4)\ni=0\nfor i in range(10):\n  qs=q3[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.05,8)\n  qs['Label']=np.arange(0,4)\n  l = l.append(qs)\n  i=i+1"

In [17]:
'''Data augmentation
q4=training (4)
i=0
for i in range(10):
  qs=q4[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.03,8)
  qs['Label']=np.arange(0,4)
  l = l.append(qs)
  i=i+1'''

"Data augmentation\nq4=training (4)\ni=0\nfor i in range(10):\n  qs=q4[['Ant1_real','Ant2_real','Ant3_real','Ant4_real','Ant1_imag','Ant2_imag','Ant3_imag','Ant4_imag']]+noise_real(0.03,8)\n  qs['Label']=np.arange(0,4)\n  l = l.append(qs)\n  i=i+1"

# Split dataset to train and test

In [18]:
'''Spliting features and labels'''
#xy=pd.concat([l,q1])       #Enable when you want vp for SVM & KNN
xy=q1                       #Enable when you dont want vp for SVM & KNN
xy_NN=pd.concat([l,q1])     #Enable when you want vp for NN
#xy_NN=q1                   #Enable when you dont want vp for NN

In [19]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.251453,-0.438900,-0.098196,0.214419,0
1,0.956773,-0.845250,0.117295,-0.142319,1
2,-0.516564,-1.211678,-0.314976,-0.205654,2
3,-0.267336,0.222052,-0.584627,-0.064228,3


#Preprocessing for KNN & SVM

In [20]:
scaler = StandardScaler()

In [21]:
scaler.fit(xy.drop('Label',axis=1))

StandardScaler()

In [22]:
scaled_features = scaler.transform(xy.drop('Label',axis=1))

In [23]:
xy = pd.DataFrame(scaled_features,columns=xy.columns[:-1])
xy['Label']=np.arange(0,4)
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.404229,0.243510,0.468808,1.645386,0
1,1.702691,-0.520325,1.297348,-0.579135,1
2,-0.866535,-1.209118,-0.364689,-0.974070,2
3,-0.431927,1.485934,-1.401467,-0.092181,3


#Preprocessing for MLP

In [24]:
xy_NN

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.235210,-0.408452,-0.083851,0.142730,0
1,0.973015,-0.814801,0.131640,-0.214008,1
2,-0.500322,-1.181229,-0.300631,-0.277342,2
3,-0.251094,0.252500,-0.570282,-0.135917,3
0,-0.232142,-0.443290,-0.087300,0.259455,0
...,...,...,...,...,...
3,-0.250102,0.206951,-0.532929,-0.073490,3
0,-0.251453,-0.438900,-0.098196,0.214419,0
1,0.956773,-0.845250,0.117295,-0.142319,1
2,-0.516564,-1.211678,-0.314976,-0.205654,2


In [25]:
'''preprocessing for NN'''
x_NN=xy_NN[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y_NN=xy_NN['Label']
'''Labels one hot encoding'''
y_NN=to_categorical(y_NN, num_classes=4)
'''Spliting data to train and validation'''
X_train_NN, X_test_NN, y_train_NN, y_test_NN = train_test_split(x_NN, y_NN, test_size=0.3, random_state=101)

In [26]:
'''Spliting features and labels for SVM & KNN'''
x=xy[['Ant1_real','Ant2_real','Ant1_imag','Ant2_imag']]
y=xy['Label']

In [27]:
xy

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag,Label
0,-0.404229,0.243510,0.468808,1.645386,0
1,1.702691,-0.520325,1.297348,-0.579135,1
2,-0.866535,-1.209118,-0.364689,-0.974070,2
3,-0.431927,1.485934,-1.401467,-0.092181,3


In [28]:
'''Spliting data to train and validation'''
#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.000001, random_state=101)
X_train=x
y_train=y

In [29]:
X_train.shape

(4, 4)

In [30]:
y_train.shape

(4,)

# Building neural network and training

In [31]:
model = Sequential()
model.add(Input(shape=(4)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))


In [32]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
NN=model.fit(X_train_NN, y_train_NN, batch_size=8, epochs=2, validation_data=(X_test_NN,y_test_NN))

Epoch 1/2
176/176 [==============================] - 3s 8ms/step - loss: 0.6805 - accuracy: 0.7561 - val_loss: 0.0555 - val_accuracy: 1.0000
Epoch 2/2
176/176 [==============================] - 1s 6ms/step - loss: 0.0546 - accuracy: 0.9929 - val_loss: 5.8928e-04 - val_accuracy: 1.0000


In [33]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 4)                 1

# Training KNN and SVM model

In [34]:
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)
knn = KNeighborsClassifier(n_neighbors=1).fit(X_train,y_train)

# Testing models on test dataset 

In [35]:
poly_pred = poly.predict(m1)
rbf_pred = rbf.predict(m1)
knn_pred = knn.predict(m1)

In [36]:
m1

,Ant1_real,Ant2_real,Ant1_imag,Ant2_imag
0,0.823210,-0.782546,0.088219,-0.084552
0,0.969688,-0.680086,-0.197416,-0.103045
0,-0.842799,-1.122123,-0.379911,-0.161689
0,-0.496119,-0.809327,-0.310262,-0.255878
0,1.118004,-0.775014,-0.491793,-0.398065
...,...,...,...,...
0,-1.047957,-0.906322,-0.257034,-0.022564
0,0.158185,-0.555106,0.025738,0.169153
0,-0.342720,-0.470870,-0.262448,-0.028559
0,-0.194493,-0.260949,-0.333626,0.443090


In [37]:
knn_accuracy = accuracy_score(n1, knn_pred)
knn_f1 = f1_score(n1, knn_pred, average='weighted')
print('Accuracy (KNN): ', "%.2f" % (knn_accuracy*100))
print('F1 (KNN): ', "%.2f" % (knn_f1*100))

Accuracy (KNN):  81.25
F1 (KNN):  80.51


In [38]:
poly_accuracy = accuracy_score(n1, poly_pred)
poly_f1 = f1_score(n1, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

Accuracy (Polynomial Kernel):  50.40
F1 (Polynomial Kernel):  36.62


In [39]:
rbf_accuracy = accuracy_score(n1, rbf_pred)
rbf_f1 = f1_score(n1, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (RBF Kernel):  81.25
F1 (RBF Kernel):  80.51


In [40]:
'''Test the model'''
NN_Accuracy=model.evaluate(m1_NN, n1_NN)

63/63 [==============================] - 0s 4ms/step - loss: 0.0363 - accuracy: 0.9885


#Choosing K value

In [41]:
'''error_rate = []

for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(m1)
    error_rate.append(np.mean(pred_i != n1))'''

'error_rate = []\n\nfor i in range(1,40):\n    knn = KNeighborsClassifier(n_neighbors=i)\n    knn.fit(X_train,y_train)\n    pred_i = knn.predict(m1)\n    error_rate.append(np.mean(pred_i != n1))'

In [42]:
'''plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)
plt.title('Error Rate vs. K Values')
plt.xlabel('K')
plt.ylabel('Error Rate')'''

"plt.figure(figsize=(10,6))\nplt.plot(range(1,40),error_rate,color='blue',ls='--',marker='o',markerfacecolor='red',markersize=10)\nplt.title('Error Rate vs. K Values')\nplt.xlabel('K')\nplt.ylabel('Error Rate')"